Performing landmark localisation when detector's/tracker's bounding box are already exported.
Added also support for detectors (e.g. ramanan) when bb's exist. 

Difference from run_online_tracking: It runs for all methods (detectors + trackers) for 
a particular landmark localisation method.

In [ ]:
from __future__ import division
from os.path import isdir, join, isfile, sep
from os import listdir 
from functools import partial
from warnings import warn
import numpy as np
from research_pyutils.path_related_functions import mkdir_p, rm_if_exists
import logging
from pathlib import Path
from workerbee import exhaust_all_files_randomly

# menpo packages imports
import menpo.io as mio
from menpodetect import load_dlib_frontal_face_detector
from menpodetect import load_opencv_frontal_face_detector
from menpodetect.ffld2 import load_ffld2_frontal_face_detector
from menpodetect.dlib.conversion import pointgraph_to_rect
from dlib import shape_predictor
from menpo.shape import PointCloud
from menpo.landmark import LandmarkGroup
from menpo.transform import Scale, Translation

In [ ]:
# print the name of the computer, useful if running on condor
import socket
import time 
print(socket.gethostname())
print(time.strftime("%d/%m/%Y, %H:%M:%S"))

In [ ]:
path_base = '/vol/atlas/homes/grigoris/misc/2016_ijcv/data/300vw_testset/'
assert(isdir(path_base))

path_pickles = '/vol/atlas/homes/grigoris/misc/2016_ijcv/data/pickles/'
assert(isdir(path_pickles))

In [ ]:
def detection_to_pointgraph(detection):
    return PointCloud(np.array([(p.y, p.x) for p in detection.parts()]))

In [ ]:
def _get_path(p0, name):
    assert(isdir(p0))
    p1 = join(p0, name, '')
    if not isdir(p1):
        mkdir_p(p1)
    return p1


def return_scale(pt, target_sz):
    # find the current range of the bb (pt) and the scaling required to reach the target_sz.
    # Used for the regression in detector's AAM only. 
    target_sz = np.array(target_sz) * 1.0
    range0 = np.max(pt, axis=0) - np.min(pt, axis=0)
    assert(np.min(range0) > 0)
    scale = target_sz / range0
    return scale


def aam_regression(im, g):
    # using global regr1, reg_f.
    # The function resizes the bb based on the regression training, that is to 
    # scale/translate the detectors' bb towards the gt bb. 
    # It initially resizes the image, extracts the feats, acquires 
    # the prediction and then applies it on the detectors' bb.
    
    # Resizes the images based on the target_sz, extracts the feats in the resized bb, 
    # fit the regression, gets the prediction and applies the transform to the image.
    # to the one the regression is trained on, extract feats
    target_sz = reg_f['target_size']
    feats = reg_f['feats']
    
    sc1 = return_scale(im.landmarks[g].lms.points, target_sz)
    im2 = im.resize(sc1 * im.shape)

    pt = im2.landmarks[g].lms.points
    min_idx = np.round(np.min(pt, axis=0))
    # often the size of the bb is 1 pixel off, so we specifically crop 
    #it to be in the target_sz.
    im3 = feats(im2.crop(min_idx, min_idx + target_sz))
    ft = np.reshape(im3.pixels, -1)
    # predict from regression method
    ft = ft.reshape(1, -1)  # avoid annoying warning of sklearn
    pred = regr1.predict(ft)
    
#     im_n = im2.resize(pred[0, 0 : 2]  * im2.shape)
#     pt = im_n.landmarks[g].lms.points
#     # translate the bb landmark group
#     im_n.landmarks['pred'] = PointCloud(pt + pred[0, 2 : 4])
    
    # get the reshaped bb, based on the prediction of the regressor.
    # WARNING: The scaling as defined in the training is a pure scaling of the 
    # width, height, while with the transform in menpo, the bounding box moves
    # towards [0,0], so we account for this change and add it to the translation.
    scale = Scale(pred[0, 0 : 2])
    centre = im.landmarks[g].lms.centre()
    pcloud = scale.apply(im.landmarks[g].lms)
    im.landmarks['pred'] = pcloud
    diff_centre = centre - im.landmarks['pred'].lms.centre()
    
    transl = Translation(pred[0, 2 : 4] + diff_centre)
    pcloud = transl.apply(im.landmarks['pred'].lms)
    # the new bb is saved as pred landmark in the original image
    im.landmarks['pred'] = pcloud
    
    return pred


# def regression_transform_back(im, pred, shape):
#     # reverse the changes made in the image
#     pt = im.landmarks['final'].lms.points
#     pt = pt - pred[0, 2 : 4]
#     im.landmarks['final'] = PointCloud(pt + pred[0, 2 : 4])
#     im2 = im.resize(shape)
#     return im2.landmarks['final']

def _area_bb(bb):
    # accepts a list with a box in the format [y0, x0, y1, x1] and returns its area
    return (bb[3] - bb[1] + 1) * (bb[2] - bb[0] + 1)
    
def compute_overlap(pt0, pt1):
    # compute the overal of two bounding boxes (voc pascal as area of intersection / area of union)
    # bbox in the form of [y0, x0, y1, x1]
    # TODO: tests -> assert that outcome is 1 if called with same input, empty bb -> 0
    b0 = [np.min(pt0[:, 0]), np.min(pt0[:, 1]), np.max(pt0[:, 0]), np.max(pt0[:, 1])]
    b1 = [np.min(pt1[:, 0]), np.min(pt1[:, 1]), np.max(pt1[:, 0]), np.max(pt1[:, 1])]
    # bounding box of intersection
    bb_i = [max(b0[0], b1[0]), max(b0[1], b1[1]), 
                       min(b0[2], b1[2]), min(b0[3], b1[3])]
    
    inter_area = _area_bb(bb_i)
    overlap = 0.
    if (bb_i[3] - bb_i[1] + 1 > 0) and inter_area > 0:
        union_area = _area_bb(b0) + _area_bb(b1) - inter_area
        overlap = inter_area / union_area
        
    return overlap
    
    
    
def process_frame(p_fr):
    # using global: p_ln_out_0, model, p_condor_dummy_0, p_bb_out_0, predictor_dlib(), min_sz
    try:
        im = mio.import_image(p_fr)
        if im.n_channels == 3:
            im = im.as_greyscale()
        p_bb_out = _get_path(p_bb_out_0, im.path.parent.name) + im.path.stem
        if isfile(p_bb_out + '.pts'):  # allow the _0.pts extension
            p_bb_out += '.pts'
        else:
            p_bb_out += '_0.pts'

        if isfile(p_bb_out):  # for this version, bb is loaded instead of detected
            ln = mio.import_landmark_file(p_bb_out)
            assert(ln.lms.n_points == 4)
            
            # catch the case of nan or overflowed values and don't try to fit those.
            if ln.lms.has_nan_values() or np.any(ln.lms.points > 1e6):
                print('The ln {} has nan or overflowed values.'.format(ln.path.stem))
                # create dummy file for workerbee
                p_cond = _get_path(p_condor_dummy_0, im.path.parent.name)
                open(p_cond + im.path.stem + '.pts', 'a').close() 
                return                

            p_out = _get_path(p_ln_out_0, im.path.parent.name) + im.path.stem + '.pts'
            # ensure that the bb is inside the bounds
            im.landmarks['bb'] = ln
            im.constrain_landmarks_to_bounds()
            ln = im.landmarks['bb']
            
            if method_landm_loc == 'dlibERT':
                im_pili = np.array(im.as_PILImage())
                det_frame = predictor_dlib(im_pili, pointgraph_to_rect(ln.lms))
                init_pc = detection_to_pointgraph(det_frame)
                mio.export_landmark_file(LandmarkGroup.init_with_all_label(init_pc), 
                                         p_out, overwrite=True)
            else:
                if ('aam' in fold_out) and ('detection' in fold_out):   # hack to load regression model 
                    pred = aam_regression(im, 'bb')
                    ov1 = compute_overlap(im.landmarks['pred'].lms.points, ln.lms.points)
                    # detectors are not too off in their prediction, so require
                    # some minimum overlap to replace with the regression's bb.
                    if ov1 > 0.45 :
                        im.constrain_landmarks_to_bounds()
                        ln = im.landmarks['pred']
                        
                # catch the rare case of almost zero-size bb, in which case menpofit might crash.
                bp = ln.lms.points
                if np.all(np.max(bp, 0) - np.min(bp, 0) >= min_sz): 
                    ft = model.fit_from_bb(im, ln.lms)
                    mio.export_landmark_file(ft.fitted_image.landmarks['final'], p_out, overwrite=True)
                    print(' successfully fitted')  # helps in condor
#                     shape = np.copy(im.shape)
#                     im_n, pred, sc1 = aam_regression(im)
#                     ln = im_n.landmarks['bb']
#                     ft = model.fit_from_bb(im_n, ln.lms)
#                     im1 = ft.fitted_image.copy()
#                     ln = regression_transform_back(im1, pred, shape)
#                     mio.export_landmark_file(ln, p_out, overwrite=True)
#                 else:
#                     ft = model.fit_from_bb(im, ln.lms)
#                     mio.export_landmark_file(ft.fitted_image.landmarks['final'], p_out, overwrite=True)
        else:  # temp, can be removed, debugging purposes. 
            print("The input '{}' has no ln.".format(p_fr))
        
        # create dummy file for workerbee
        p_cond = _get_path(p_condor_dummy_0, im.path.parent.name)
        open(p_cond + im.path.stem + '.pts', 'a').close() 
    except Exception as e:
        print("The input '{}' failed with p_ln '{}'.".format(p_fr, p_out))
        logging.exception("The input '{}' failed.".format(p_fr))
        if ('aam' in fold_out) and ('detection' in fold_out):   # hack to load regression model 
            try:
                print(e)
                # create dummy file for workerbee
                p_cond = _get_path(p_condor_dummy_0, im.path.parent.name)
                open(p_cond + im.path.stem + '.pts', 'a').close() 
            except:
                pass
        else:
            raise ValueError()

In [ ]:
min_sz = np.array([10, 10])  # minimum size of bb we 'allow'.
method_landm_loc = 'aam'

detections_methods = ['dlib', 'opencv', 'ffld2', 'ramanan']
tracking_methods = ['corr', 'fct', 'rpt', 'lrst', 'spot', 'tld', 'srdcf', 
                   'kcf', 'cmt', 'mil']
# detectors' pool
fold_out_pool_0 = ['detection_' + i + '_' + method_landm_loc for i in detections_methods]
f0 = ['detector_' + i for i in detections_methods]
# trackers' pool
fold_out_pool_1 = ['tracking_' + i + '_' + method_landm_loc for i in tracking_methods]
f1 = ['tracker_' + i  for i in tracking_methods]
# combine the two lists
f_o_pool = fold_out_pool_0 + fold_out_pool_1
f_d_o_pool = f0 + f1
names = detections_methods + tracking_methods
assert(len(f_d_o_pool) == len(f_o_pool))

In [ ]:
cats = listdir(path_base)

for f0 in range(len(f_o_pool)):
    fold_out = f_o_pool[f0]
    fold_det_out = f_d_o_pool[f0]
    print('method: {}, detector: {}'.format(fold_out, fold_det_out))

    if fold_det_out[:7] == 'tracker':
        model_bb = 'gt_bb'
    elif fold_det_out[:8] == 'detector':
        model_bb = names[f0]
    else:
        raise ValueError('Not a valid option')
    
    # load prediction model
    if method_landm_loc == 'dlibERT':
        path_shape_pred = path_pickles + 'modelln_' +  model_bb + '_' + method_landm_loc + '.model'
        assert(isfile(path_shape_pred))
        predictor_dlib = shape_predictor(path_shape_pred)
        model = 'dummy'  # just for deleting the models in the else below
    else:
        path_pkl = path_pickles + 'modelln_' +  model_bb + '_' + method_landm_loc + '.pkl'
        assert(isfile(path_pkl))
        model = mio.import_pickle(path_pkl)
        
    if ('aam' in fold_out) and ('detection' in fold_out):  # hack for detector's AAM.
        p_regr = join(path_pickles, 'regressor_' + fold_det_out + '.pkl')
        assert(isfile(p_regr))
        reg_f = mio.import_pickle(p_regr)
        regr1 = reg_f['regr']
        
    # for each category in the testset, run landmark loc method
    for cat in cats:
        if not cat[:8] == 'category' or not isdir(path_base + cat):
            warn('Unknown content in path {} (folder: {}).'.format(path_base, cat))
        print(cat)
        # join or create the paths
        p_cat = join(path_base, cat, '')
        p_fr = join(p_cat, 'frames', '')
        p_ln_out_0 = mkdir_p(join(p_cat, fold_out, ''))
        p_bb_out_0 = mkdir_p(join(p_cat, fold_det_out, ''))
        p_condor_dummy_0 = mkdir_p(join(p_cat, 'condor_tmp', 'condor_dummy_' + fold_out, ''))
        assert(isdir(p_fr))  # frames folder should exist

        for c in sorted(listdir(p_fr)):   # for each clip
            output_dir = Path(mkdir_p(p_condor_dummy_0 + c + sep))
            done = lambda: output_dir.glob('*.pts')
            im_paths = lambda: mio.image_paths(p_fr + c + '/*')
            exhaust_all_files_randomly(im_paths, done, process_frame, verbose=True)
        # uncomment elow, only if NOT called in condor.
    #     rm_if_exists(p_condor_dummy_0)
    del model


In [ ]:
import time 
print()
print(time.strftime("%d/%m/%Y, %H:%M:%S"))